In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
from itertools import permutations

import openai
from dotenv import load_dotenv

import art

# from art.local import LocalBackend

load_dotenv()

True

In [4]:
import os

openai_client = openai.AsyncOpenAI(
    api_key=os.getenv("WANDB_API_KEY"),
    base_url="https://api.inference.coreweave.com/v1",
)
await openai_client.chat.completions.create(
    messages=[{"role": "user", "content": "Hello, world!"}],
    model="Qwen/Qwen2.5-14B-Instruct",
)

ChatCompletion(id='chatcmpl-16c4386e-d10f-4d6c-a237-75b4c8813c56', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! It's nice to meet you. How can I assist you today?", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1759795924, model='Qwen/Qwen2.5-14B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=33, total_tokens=50, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None, kv_transfer_params=None)

In [5]:
import wandb

run = wandb.init(project="es")

wandb: Currently logged in as: bhilton (wandb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Weave is installed but not imported. Add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [6]:
from typing import cast

import peft
import unsloth
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.utils.dummy_pt_objects import (
    GenerationMixin,
    PreTrainedModel,
)


class CausallLM(PreTrainedModel, GenerationMixin):
    pass


model_id = "Qwen/Qwen2.5-14B-Instruct"
is_moe = False
model, tokenizer = cast(
    tuple[CausallLM, PreTrainedTokenizerBase],
    (unsloth.FastModel if is_moe else unsloth.FastLanguageModel).from_pretrained(
        model_name=model_id,
        load_in_4bit=False,
        max_seq_length=32768,  # Qwen2.5 supports up to 32k context
    ),
)
peft_model = cast(
    peft.peft_model.PeftModelForCausalLM,
    unsloth.FastModel.get_peft_model(
        model,
        lora_alpha=16,
        r=8,
        random_state=3407,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
        + (["gate_proj", "up_proj", "down_proj"] if not is_moe else []),
    ),
)
lora_model = cast(peft.tuners.lora.LoraModel, peft_model.base_model)

/tmp/ipykernel_1100204/2800122005.py:4: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-07 00:12:13 [__init__.py:235] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.6: Fast Qwen2 patching. Transformers: 4.53.2. vLLM: 0.10.0.
   \\   /|    NVIDIA H200. Num GPUs = 2. Max memory: 139.811 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Unsloth: Making `model.base_model.model.model` require gradients


In [7]:
peft_model.save_pretrained("./lora")

In [8]:
import asyncio
import os
import shutil
import tempfile
import time

import torch
import wandb
from safetensors.torch import load_file, save_file


async def copy_lora_add_noise_and_log(
    source_dir: str = "./lora",
    noise_std: float = 1e-4,
    artifact_type: str = "lora",
    run=None,
) -> str:
    """Copy ./lora to /tmp, add Gaussian noise to adapter weights on CPU, save, and log as a W&B artifact.
    Returns the destination directory path.
    """
    dst = tempfile.mkdtemp(prefix="lora_", dir="/tmp")
    print(f"Copying {source_dir} -> {dst}")
    shutil.copytree(source_dir, dst, dirs_exist_ok=True)

    safetensors_path = os.path.join(dst, "adapter_model.safetensors")
    tensors = load_file(safetensors_path, device="cpu")

    num_noised = 0
    total = 0
    with torch.inference_mode():
        for _, tensor in tensors.items():
            total += tensor.numel()
            if tensor.is_floating_point():
                tensor.add_(torch.randn_like(tensor) * noise_std)
                num_noised += tensor.numel()

    save_file(tensors, safetensors_path)

    run = run or wandb.run
    if run is not None:
        name = f"lora-noised-{int(time.time())}"
        artifact = wandb.Artifact(
            name=name,
            type=artifact_type,
            metadata={
                "noise_std": float(noise_std),
                "wandb.base_model": "Qwen/Qwen2.5-14B-Instruct",
            },
            storage_region="coreweave-us",
        )
        artifact.add_dir(dst)
        run.log_artifact(artifact)
        await asyncio.to_thread(artifact.wait)
        print(
            f"Noised {num_noised}/{total} params; saved to {safetensors_path}; logged artifact '{name}'."
        )
    else:
        print(
            f"Noised {num_noised}/{total} params; saved to {safetensors_path}; no active W&B run found."
        )

    return dst

In [9]:
await copy_lora_add_noise_and_log()

Copying ./lora -> /tmp/lora_tiswiiol


wandb: Adding directory to artifact (/tmp/lora_tiswiiol)... Done. 0.2s


Noised 34406400/34406400 params; saved to /tmp/lora_tiswiiol/adapter_model.safetensors; logged artifact 'lora-noised-1759796163'.


'/tmp/lora_tiswiiol'

In [13]:
entity = run.entity
project = run.project
name = run.name
(entity, project, name)

('wandb', 'es', 'northern-paper-4')

In [16]:
await openai_client.chat.completions.create(
    messages=[{"role": "user", "content": "Hello, world!"}],
    model=f"wandb-artifact:///{entity}/{project}/lora-noised-1759796163:v0",
)

ChatCompletion(id='chatcmpl-496685f6-1a6e-4027-8c20-250b3749612d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! It's nice to meet you. How can I assist you today?", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1759796917, model='wandb-artifact:///wandb/es/lora-noised-1759796163:v0', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=33, total_tokens=50, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None, kv_transfer_params=None)